In [192]:
#Importamos las librerías que vamos a necesitar
import pandas as pd
import numpy as np
import sys
import os
import warnings
warnings.filterwarnings('ignore')

In [193]:
df_tablas = pd.read_csv(f"../datos/df_supermercados.csv")

## Preparación de las tablas para la base de datos

### Tabla productos

In [194]:
df_productos = pd.DataFrame(df_tablas["categoria_producto"].unique())
df_productos.reset_index(inplace=True)
df_productos.rename(columns={'index':'id_producto', 0:'nombre'}, inplace=True)
df_productos["id_producto"]=df_productos["id_producto"].apply(lambda x: x+1)
df_productos

,id_producto,nombre
0,1,leche


In [195]:
# df_productos.to_csv("../datos/tabla_productos.csv")

In [196]:
lista_tuplas_productos = []
for fila in df_productos.values:
    lista_tuplas_productos.append(tuple(fila))

### Tabla supermercados

In [197]:
df_superm = pd.DataFrame(df_tablas["supermercado"].unique())
df_superm.reset_index(inplace=True)
df_superm.rename(columns={'index':'id_super', 0:'nombre'}, inplace=True)
df_superm["id_super"]=df_superm["id_super"].apply(lambda x: x+1)
df_superm

,id_super,nombre
0,1,alcampo
1,2,carrefour
2,3,dia
3,4,eroski
4,5,hipercor
5,6,mercadona


In [198]:
# df_superm.to_csv("../datos/tabla_superm.csv")

In [199]:
lista_tuplas_supermercados = []
for fila in df_superm.values:
    lista_tuplas_supermercados.append(tuple(fila))

### Tabla histórico

Incluimos los id del supermercado y producto que serán las claves foraneas:

In [200]:
df_aux_historico = df_tablas.merge(right=df_superm, how='left', left_on ='supermercado', right_on="nombre")
df_aux_historico=df_aux_historico.drop(columns=["Unnamed: 0", "supermercado", "nombre"])
df_aux_historico_2 = df_aux_historico.merge(right=df_productos, how='left', left_on ='categoria_producto', right_on="nombre")
df_aux_historico_2 = df_aux_historico_2.drop(columns=["categoria_producto", "nombre"])

Reordenamos y borramos duplicados

In [201]:
df_aux_historico_2 = df_aux_historico_2.reindex(columns=['id_super', 'id_producto', 'nombre_producto', 'subcategoria', 'Día', 'Precio (€)', 'cantidad_variacion', 'porcentaje_variacion'])
df_aux_historico_2.drop_duplicates(inplace=True)
df_historico = df_aux_historico_2
df_historico.sample(5)

,id_super,id_producto,nombre_producto,subcategoria,Día,Precio (€),cantidad_variacion,porcentaje_variacion
133073,5,1,la masia aceite de girasol garrafa de 5 l,NaN,2024-07-28,15.88,0.0,0.0
62595,2,1,priegola leche entera fresca 1 l,entera,2024-10-10,2.20,0.0,0.0
24455,1,1,lr leche semidesnatada de vaca sin lactosa 1l,semidesnatada,2024-09-21,0.87,0.0,0.0
29133,1,1,president leche entera de vaca de origen 100 e...,entera,2024-09-06,1.45,0.0,0.0
34283,2,1,producto alcampo aceite de girasol botella de 1 l,NaN,2024-07-08,1.45,0.0,0.0


In [202]:
# df_historico.to_csv("../datos/tabla_historico.csv")

In [203]:
lista_tuplas_historico = []
for fila in df_historico.values:
    lista_tuplas_historico.append(tuple(fila))